# 如何筛选消息
在更复杂的链和代理中，我们可能会通过[消息](/docs/concepts/messages/)列表来跟踪状态。这个列表可能开始累积来自多个不同模型、发言者、子链等的消息，而我们可能只希望将完整消息列表中的子集传递给链/代理中的每个模型调用。
`filter_messages` 工具可以轻松地按类型、ID 或名称筛选消息。
## 基本用法

In [1]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    filter_messages,
)

messages = [
    SystemMessage("you are a good assistant", id="1"),
    HumanMessage("example input", id="2", name="example_user"),
    AIMessage("example output", id="3", name="example_assistant"),
    HumanMessage("real input", id="4", name="bob"),
    AIMessage("real output", id="5", name="alice"),
]

filter_messages(messages, include_types="human")

[HumanMessage(content='example input', name='example_user', id='2'),
 HumanMessage(content='real input', name='bob', id='4')]

In [2]:
filter_messages(messages, exclude_names=["example_user", "example_assistant"])

[SystemMessage(content='you are a good assistant', id='1'),
 HumanMessage(content='real input', name='bob', id='4'),
 AIMessage(content='real output', name='alice', id='5')]

In [3]:
filter_messages(messages, include_types=[HumanMessage, AIMessage], exclude_ids=["3"])

[HumanMessage(content='example input', name='example_user', id='2'),
 HumanMessage(content='real input', name='bob', id='4'),
 AIMessage(content='real output', name='alice', id='5')]

## 链式调用
`filter_messages` 既可以通过命令式（如上所示）也可以通过声明式的方式使用，从而轻松与其他链式组件组合：

In [ ]:
%pip install -qU langchain-anthropic

In [4]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)
# Notice we don't pass in messages. This creates
# a RunnableLambda that takes messages as input
filter_ = filter_messages(exclude_names=["example_user", "example_assistant"])
chain = filter_ | llm
chain.invoke(messages)

AIMessage(content=[], response_metadata={'id': 'msg_01Wz7gBHahAwkZ1KCBNtXmwA', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 16, 'output_tokens': 3}}, id='run-b5d8a3fe-004f-4502-a071-a6c025031827-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

查看LangSmith跟踪记录可以发现，在消息传递给模型之前会经过过滤处理：https://smith.langchain.com/public/f808a724-e072-438e-9991-657cc9e7e253/r
仅观察`filter_`部分，我们可以看出它是一个可运行的（Runnable）对象，能够像所有可运行对象一样被调用：

In [6]:
filter_.invoke(messages)

[HumanMessage(content='real input', name='bob', id='4'),
 AIMessage(content='real output', name='alice', id='5')]

## API 参考
要查看所有参数的完整描述，请前往API参考文档：[https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.filter_messages.html](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.filter_messages.html)